In [16]:
# importing the libraries required

import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", message="^WARNING:tensorflow:")

In [17]:
# path of training data directory

train_dir = '/kaggle/input/face-mask-dataset/FMD_DATASET'

In [18]:
# generate the data from training directory

dataGenerator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, horizontal_flip=True, validation_split=0.2)

training_data = dataGenerator.flow_from_directory(train_dir, batch_size=64, target_size=(48, 48), shuffle=True, color_mode='grayscale', class_mode='categorical', subset='training')
validation_set = dataGenerator.flow_from_directory(train_dir, batch_size=64, target_size=(48, 48), shuffle=True, color_mode='grayscale', class_mode='categorical', subset='validation')


Found 11630 images belonging to 3 classes.
Found 2906 images belonging to 3 classes.


In [19]:
# creating model 

def create_model():
    weight_decay = 1e-4
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Conv2D(64, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay), input_shape=(48, 48, 1)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(256, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.4))
    
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation="linear"))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    return model

In [20]:
# compiling the model 

model = create_model()

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [21]:
# creating checkpointer

checkpointer = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', verbose = 1, restore_best_weights=True, mode="max",patience = 10),
                tf.keras.callbacks.ModelCheckpoint(
                    filepath='/kaggle/working/model-{epoch}.hdf5',
                    monitor="val_accuracy",
                    verbose=1,
                    mode="max")]

In [ ]:
# fitting the model


steps_per_epoch = training_data.n // training_data.batch_size
validation_steps = validation_set.n // validation_set.batch_size

history = model.fit(x=training_data,
                    validation_data=validation_set,
                    epochs=100,
                    callbacks=[checkpointer],
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps)

Epoch 1/100


2023-06-13 05:33:14.240906: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


 43/181 [======>.......................] - ETA: 49s - loss: 0.6992 - accuracy: 0.7620

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


181/181 [==============================] - ETA: 0s - loss: 0.3828 - accuracy: 0.8821
Epoch 1: saving model to /kaggle/working/model-1.hdf5
181/181 [==============================] - 86s 454ms/step - loss: 0.3828 - accuracy: 0.8821 - val_loss: 1.1991 - val_accuracy: 0.3281
Epoch 2/100
181/181 [==============================] - ETA: 0s - loss: 0.1889 - accuracy: 0.9496
Epoch 2: saving model to /kaggle/working/model-2.hdf5
181/181 [==============================] - 91s 502ms/step - loss: 0.1889 - accuracy: 0.9496 - val_loss: 2.1502 - val_accuracy: 0.3267
Epoch 3/100
181/181 [==============================] - ETA: 0s - loss: 0.1541 - accuracy: 0.9646
Epoch 3: saving model to /kaggle/working/model-3.hdf5
181/181 [==============================] - 88s 485ms/step - loss: 0.1541 - accuracy: 0.9646 - val_loss: 1.3969 - val_accuracy: 0.4170
Epoch 4/100
181/181 [==============================] - ETA: 0s - loss: 0.1383 - accuracy: 0.9705
Epoch 4: saving model to /kaggle/working/model-4.hdf5
181/18